In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
# setup cell to make our lives easier 
import re
import numpy as np
import os
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
from collections import defaultdict
from utils import *
from customize_dataset import DexNetNPZDataset
from customize_dataset import DexNetNPZDatasetAll

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.amp import GradScaler, autocast

tensor_dir = '../../dexnet_2.1/dexnet_2.1_eps_10/tensors/'  # replace with actual path
batch_size = 32
use_regression = False  # or True
pose_dims = [2]  


In [2]:
# setup block. Run me!
print(torch.cuda.device_count())
print(torch.cuda.get_device_name())
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

1
NVIDIA GeForce RTX 3060 Laptop GPU


In [7]:
def evaluate_accuracy_with_confusion(model, dataloader, device, threshold=0.5):
    model.eval()
    correct = 0
    total = 0

    # Initialize confusion matrix counts
    TP = FP = TN = FN = 0

    with torch.no_grad():
        for images, poses, labels in dataloader:
            images = images.to(device, non_blocking=True).contiguous(memory_format=torch.channels_last)
            poses = poses.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True).unsqueeze(1)

            outputs = model(images, poses)
            outputs = torch.sigmoid(outputs)
            predictions = (outputs >= threshold).float()

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

            # Compute confusion matrix components
            TP += ((predictions == 1) & (labels == 1)).sum().item()
            TN += ((predictions == 0) & (labels == 0)).sum().item()
            FP += ((predictions == 1) & (labels == 0)).sum().item()
            FN += ((predictions == 0) & (labels == 1)).sum().item()

    accuracy = correct / total

    # Normalize confusion matrix to percentage
    percent = lambda x: (x / total) * 100
    confusion_matrix = [
        [percent(TN), percent(FP)],  # row for actual 0
        [percent(FN), percent(TP)]   # row for actual 1
    ]

    print(f"✅ Accuracy: {accuracy * 100:.2f}%")
    print("📊 Confusion Matrix (in %):")
    print(f"            Pred 0     Pred 1")
    print(f"Actual 0   {confusion_matrix[0][0]:6.2f}%   {confusion_matrix[0][1]:6.2f}%")
    print(f"Actual 1   {confusion_matrix[1][0]:6.2f}%   {confusion_matrix[1][1]:6.2f}%")

    return accuracy, confusion_matrix

In [3]:
class SimpleGQCNN(nn.Module):
    def __init__(self, pose_dim=4, output_type='binary', merge_methods="element_dot"):
        """
        pose_dim: number of dimensions in the pose vector (e.g., x, y, z, theta)
        output_type: 'binary' or 'regression'
        """
        super(SimpleGQCNN, self).__init__()
        self.output_type = output_type

        # Image stream
        self.conv1 = nn.Conv2d(1, 16, 3)           # → (B, 16, 30, 30)
        self.pool = nn.MaxPool2d(2, 2)             # → (B, 16, 15, 15)
        self.conv2 = nn.Conv2d(16, 32, 3)          # → (B, 32, 13, 13) → pool → (B, 32, 6, 6)
        self.im_fc = nn.Linear(32 * 6 * 6, 64)     # → (B, 64)
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.im_fc_bn = nn.BatchNorm1d(64)

        # Pose stream
        self.pose_fc1 = nn.Linear(pose_dim, 64)
        self.pose_fc2 = nn.Linear(64, 64)
        self.dropout = nn.Dropout(p=0.5)
        self.merge_methods = merge_methods
        if self.merge_methods == "element_dot":
            # Merge stream after elementwise multiplication
            self.merge_fc1 = nn.Linear(64, 32)
            self.merge_fc2 = nn.Linear(32, 1)
        else:
            # Merge stream by concatanation
            self.merge_fc1 = nn.Linear(64 + 64, 64)
            self.merge_fc2 = nn.Linear(64, 1)  # Single output for binary or regression

    def forward(self, image, pose):
        
        # Image stream
        x = self.pool(F.relu(self.bn1(self.conv1(image))))   # (B, 16, 15, 15)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))       # (B, 32, 6, 6)
        # x = x.view(x.size(0), -1)                  # Flatten
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.im_fc(x))                  # (B, 64)

        # Pose stream
        p = self.dropout(F.relu(self.pose_fc1(pose)))            # (B, 64)
        p = self.dropout(F.relu(self.pose_fc2(p)))               # (B, 64)

        if self.merge_methods == "element_dot":
            # Element-wise multiplication
            combined = x * p                           # (B, 64)
        else:
            # Merge
            combined = torch.cat((x, p), dim=1)       # -> (B, 96)

        # Final layers
        out = F.relu(self.merge_fc1(combined))     # (B, 32)
        out = self.merge_fc2(out)                  # (B, 1)

        # if self.output_type == 'binary':
        #     out = torch.sigmoid(out)               # Binary prediction
        return out 


In [6]:
# load SimpleGQCNN
pose_dims = [0, 1, 2, 3, 4, 5]
model = SimpleGQCNN(pose_dim=len(pose_dims), output_type='regression' if use_regression else 'binary')
model.load_state_dict(torch.load("eps_10/model.pth", weights_only=True))
model = model.to(device, memory_format=torch.channels_last)

In [10]:
import itertools
# Create dataset and dataloader
train_loader, val_loader = DexNetDataloader(tensor_dir=tensor_dir, use_regression=use_regression, pose_dims=pose_dims)
 
subset_loader = itertools.islice(val_loader, 5)  # Use only the first 5 batches

evaluate_accuracy_with_confusion(model, subset_loader, device, threshold=0.5)

✅ Accuracy: 92.50%
📊 Confusion Matrix (in %):
            Pred 0     Pred 1
Actual 0     3.75%     5.62%
Actual 1     1.88%    88.75%


(0.925, [[3.75, 5.625], [1.875, 88.75]])